# **Vector Search Fundamentals**

We are now transitioning from generating embeddings (which we will cover more deeply in M3) to **managing and searching** them at scale. This is the backbone of RAG and modern retrieval systems.

We will focus on the fundamental geometry of vector spaces and how to traverse them efficiently using the **FAISS** library.

Here is the breakdown for today's session.

### Phase 1: Topic Breakdown

```text
L17: Vector Search Fundamentals
├── Concept 1: Vector Spaces & High-Dimensional Data
│   ├── Embeddings as coordinates
│   ├── The Matrix (N vectors x D dimensions)
│   ├── Simple Explanation: Finding points in a massive hyper-cube
│   └── Task: Generate synthetic vector dataset (NumPy)
│
├── Concept 2: Distance Metrics
│   ├── Euclidean Distance (L2) - Physical distance
│   ├── Cosine Similarity - Angular distance
│   ├── Intuition: Magnitude vs. Orientation
│   └── Task: Manual calculation of metrics using NumPy
│
├── Concept 3: ANN Theory (Approximate Nearest Neighbors)
│   ├── The Scaling Problem (O(N) Complexity)
│   ├── IVF (Inverted File Index) - Partitioning/Clustering
│   ├── HNSW (Hierarchical Navigable Small World) - Graph Traversal
│   └── Task: Conceptual Check (Trade-offs)
│
├── Concept 4: FAISS Basics (The Tool)
│   ├── What is FAISS? (Facebook AI Similarity Search)
│   ├── The Index Object (IndexFlatL2)
│   ├── The Workflow: Index -> Add -> Search
│   └── Task: Implement "Brute Force" search in FAISS
│
└── Mini-Project: The Search Benchmark
    ├── Setup Large Dataset
    ├── Implement IndexFlatL2 (Exact)
    ├── Implement IndexIVFFlat (Approximate)
    ├── Train the Index (Clustering)
    └── Compare speed (latency) and recall

```

---


## **Concept 1: Vector Spaces & High-Dimensional Data**

### Intuition

In traditional databases, we search for exact matches (e.g., `SELECT * FROM users WHERE name = "Alice"`). However, in AI, we often want to search for *meaning*. To do this, we convert complex data like text or images into lists of numbers called **vectors** (or embeddings).

Imagine a 2D graph. A point at `(2, 3)` is a vector. Now, imagine a graph with 128, 768, or even 1536 axes. This is a **high-dimensional vector space**. Every piece of data becomes a single point in this space. Data that is semantically similar (e.g., "Dog" and "Puppy") will be located physically close to each other in this space.

### Mechanics

We represent this data mathematically as a Matrix $X$ of size $N \times D$ :
   * **$N$**: The number of samples (vectors) in your database.
   * **$D$**: The dimensionality of each vector (determined by the model, e.g., BERT uses 768).

In Python/NumPy, this is simply a 2D array. Crucially, most vector search libraries (including FAISS) are highly optimized for `float32` data types. Using `float64` (default in Python) can cause errors or unnecessary memory bloat.

### Simpler Explanation

Imagine a massive library. Instead of organizing books by genre or title, we give every book a precise GPS coordinate in a multi-dimensional universe. If you want a book about "Cooking," you don't look up the word; you go to the "Cooking" coordinates, and you'll find all the relevant books clustered right there.

### Trade-offs
   * **Pros:** Allows semantic search (matching meaning, not just keywords).
   * **Cons:** "The Curse of Dimensionality." As $D$ increases, the amount of data needed to generalize increases, and calculating distances becomes computationally expensive.

---

### Your Task

You need to simulate a dataset of embeddings to prepare for our search algorithms.

**Specifications:**
   1. Create a Python script using `numpy`.
   2. Define two constants: `nb` (number of database vectors) = 10,000 and `d` (dimension) = 128.
   3. Generate a matrix `xb` of shape `(nb, d)` filled with random numbers.
   4. **Critical:** Ensure the data type of the matrix is explicitly `float32`.
   5. Set a random seed so our results are reproducible.
   6. Print the shape and data type of `xb` to verify.


In [ ]:
import numpy as np

np.random.seed(42)
nb = 10000
d = 128

xb = np.random.rand(nb, d).astype(np.float32)
xb.shape

## **Concept 2: Distance Metrics**

### Intuition

Once we have points in space, we need a ruler to measure how close they are. In AI, "closeness" implies similarity. If the distance between the "User Query" vector and a "Document" vector is small, that document is likely relevant.

### Mechanics

There are two primary ways to measure this in high-dimensional spaces:

1. **Euclidean Distance (L2):**
   * The straight-line distance between two points.
   * Formula: $d(x, y) = \sqrt{\sum_{i=1}^{D} (x_i - y_i)^2}$
   * **Behavior:** Sensitive to the magnitude (length) of vectors. If one vector is  and another is , they are far apart even if they point in the same direction.

2. **Inner Product (Dot Product) / Cosine Similarity:**
   * Measures the alignment (angle) between vectors.
   * Formula (Dot Product): $IP(x, y) = \sum_{i=1}^{D} x_i \cdot y_i$
   * **Behavior:** If vectors are normalized (length = 1), Inner Product *is* Cosine Similarity. It focuses on "direction" rather than "magnitude."



### Simpler Explanation
   * **Euclidean:** How much physical energy does it take to walk from point A to point B?
   * **Cosine:** Are we pointing at the same star? (It doesn't matter if you are standing on Earth or Mars; if you both point at the sun, the angle difference is low).

### Trade-offs
   * **L2:** Good when magnitude matters (e.g., pixel intensity in images).
   * **Inner Product:** Usually preferred for text/semantic search (NLP) because the length of the document text shouldn't necessarily penalize the match. **FAISS** is highly optimized for L2 and Inner Product.

---

### Your Task

You will implement these metrics manually using NumPy to understand the linear algebra behind the scenes.

**Specifications:**
   1. Generate a **query matrix** `xq` containing **5 vectors** of dimension `d` (use `np.random`).
   2. Ensure `xq` is `float32`.
   3. **Task A (L2 Distance):** Calculate the squared Euclidean distance between the **first vector** of `xq` and the **first vector** of `xb`. Use the formula: `sum((x - y)^2)`.
   4. **Task B (Dot Product):** Calculate the dot product between the **first vector** of `xq` and the **first vector** of `xb`.
   5. Print both results.

**Constraints:**
   * Do not use `scipy` or `sklearn`. Use standard NumPy math.
   * Do not perform matrix multiplication for the whole dataset yet; just do 1-to-1 vector comparison for this step.

In [ ]:
import numpy as np

xq = np.random.rand(5, d).astype(np.float32)

x = xb[0]
y = xq[0]

dist = np.sum((x - y) ** 2)
dot_p = np.dot(x, y)

print(f"Distance = {dist}\nDot Product = {dot_p}")

The output shows a non-zero distance ($22.06$), confirming that `xq` and `xb` are now distinct vectors in the space.

---

## **Concept 3: ANN Theory (Approximate Nearest Neighbors)**

### Intuition

In your previous task, you compared 1 query vector against 1 database vector. To find the "nearest neighbor" for a query, you'd have to compare it against **all** 10,000 vectors in `xb`.

If $N = 10,000$, that's fine.
If $N = 1 \text{ Billion}$ (web scale), calculating 1 Billion distances for *every single user query* is impossible in real-time.

**ANN (Approximate Nearest Neighbors)** algorithms solve this by trading a tiny bit of accuracy (Recall) for massive speed (Latency). We accept that we might find the 2nd or 3rd closest match instead of the absolute 1st, in exchange for searching only a fraction of the data.

### Mechanics: Two Main Approaches

1. **IVF (Inverted File Index): Partitioning**
   * We divide the vector space into clusters (cells).
   * We compute the "centroid" (center point) of each cluster.
   * When a query comes in, we measure the distance to the *centroids*.
   * We identify the closest centroid and **only** search the vectors inside that specific cell.
   * **Analogy:** Instead of checking every book in the library, check the catalog to find the "Cooking" section, then search only that shelf.


2. **HNSW (Hierarchical Navigable Small World): Graph Traversal**
   * Vectors are nodes in a graph connected to their neighbors.
   * It builds a multi-layer graph (like a highway system).
   * Top layers have long connections (fast travel across the map). Bottom layers have dense connections (local precision).
   * **Analogy:** Playing "Six Degrees of Kevin Bacon" to find a path from A to B.



### Simpler Explanation
   * **Brute Force (Flat):** Check every single haystalk to find the needle. (100% accurate, Slow).
   * **ANN (IVF/HNSW):** Use a metal detector to find the general area, then look there. (99% accurate, Super Fast).

### Trade-offs
   * **Brute Force:** Recall = 100%. Latency = High. Memory = Low.
   * **IVF:** Recall = Tunable (based on how many cells you probe). Latency = Low. Requires "Training" step.
   * **HNSW:** Recall = High. Latency = Very Low. Memory = High (needs to store graph edges).

---

### Your Task

This is a conceptual check to ensure you understand the architecture choices before we code.

**Scenario:**

1. **System A:** A FaceID unlock system for a secure building. The database has only 500 employees. Accuracy must be perfect.
2. **System B:** An e-commerce recommendation engine ("Similar products"). The database has 50 million items. Speed is critical; if the user waits >200ms, they leave.

**Question:** Which approach (Brute Force vs. ANN) would you choose for System A and System B?



   * **System A:** Brute force is best because  is trivial for a computer (microseconds), and security requires 100% accuracy.
   * **System B:** ANN is required because searching 50M vectors linearly would crash the latency requirements (seconds/minutes), and "good enough" recommendations are acceptable.

---

## **Concept 4: FAISS Basics (The Tool)**

### Intuition

**FAISS (Facebook AI Similarity Search)** is the industry-standard library for efficient similarity search. It wraps complex C++ optimization algorithms in a simple Python interface.

The core object in FAISS is the **Index**.
Think of the Index as a specialized container. You pour your vectors into it, and it organizes them for search. Different types of Indices represent different algorithms (Brute Force, IVF, HNSW, etc.).

### Mechanics: The "Flat" Index (Brute Force)

The simplest index is `IndexFlatL2`.
   1. **Instantiation:** You create the index by telling it the dimension  of your vectors.
   2. **Adding Data:** You pass your database matrix (`xb`) to the index. It stores them in memory.
   3. **Searching:** You pass your query matrix (`xq`) and a number `k` (how many neighbors you want).
   4. **Output:** It returns two matrices:
      * **D (Distances):** The actual squared L2 distances to the neighbors.
      * **I (Indices):** The row IDs (0 to N-1) of the neighbors found in `xb`.



### Simpler Explanation
   * **Index:** The librarian.
   * **add():** Giving the librarian the books to put on the shelf.
   * **search():** Asking the librarian, "Give me the IDs of the 4 books closest to this one."

### Trade-offs
   * **IndexFlatL2** is "Exact Search." It does not compress vectors.
   * **Pros:** Perfect accuracy. Simple to set up.
   * **Cons:** Stores all vectors in RAM uncompressed. Slows down linearly as data grows.

---

### Your Task

Implement a "Brute Force" exact search using FAISS.

**Specifications:**
   1. Import `faiss` (ensure you have `faiss-cpu` installed).
   2. Create an `IndexFlatL2` object. It requires the dimension `d` as an argument.
   3. Add your database vectors `xb` to the index.
   4. Perform a search using `xq` (your 5 query vectors). Set `k = 4` (find the 4 nearest neighbors for each query).
   5. Print the resulting **Indices (`I`)** and **Distances (`D`)**.
   6. **Verify:** Look at the output. For the very first query vector, does the distance match the one you manually calculated in Concept 2? (It should match exactly).


In [ ]:
import faiss
import numpy as np

np.random.seed(42)
nb = 10000
nq = 5
d = 128

xb = np.random.rand(nb, d).astype(np.float32)
xq = np.random.rand(nq, d).astype(np.float32)

index = faiss.IndexFlatL2(d)

index.add(xb)
k = 4

D, I = index.search(xq, k)

print("Indices (I):")
print(I)

print("\nDistances (D):")
print(D)

**Verification:**
In your manual calculation (Concept 2, second attempt), you got a distance of approximately **22.06** for `xq[0]`.
Here, the closest neighbor for `xq[0]` (Index 8769) has a distance of **13.35**.

**Why the difference?**
In Concept 2, we calculated the distance between `xq[0]` and `xb[0]` (a random vector).
Here, FAISS searched the *entire* database of 10,000 vectors and found the one that was actually closest (Index 8769). Naturally, the distance to the "best match" (13.35) is smaller than the distance to a "random match" (22.06). This proves the search worked!

---


## **Mini-Project: The Search Benchmark**

The industry standard for benchmarking vector search is the **SIFT1M** (1 Million SIFT vectors) dataset.

* **Link:** [Corpus-Texmex (SIFT1M)](https://www.google.com/search?q=http://corpus-texmex.irisa.fr/)
* **Note:** Real-world datasets like SIFT1M require complex binary parsing (`.fvecs` format) which distracts from our algorithm learning.

For this project, we will generate a **Synthetic Dataset** of **100,000 vectors**. This simulates the *scale* of real data (where FAISS shines) without the overhead of downloading/parsing 1GB files.

**Objective:** Quantify the trade-off between **Accuracy (Recall)** and **Speed (Latency)** by implementing both Exact and Approximate search.

### Specifications:
   1. **Dataset Setup:**
      * **Dimension ($d$):** 128
      * **Database Size ($nb$):** 100,000 (Float32)
      * **Query Size ($nq$):** 100 (Float32)
      * **Seed:** 42 (Reproducibility)
   
   
   2. **Experiment A: Exact Search (The Baseline)**
      * Build an `IndexFlatL2`.
      * Add `xb` (database).
      * Search for `k=1` neighbor for all query vectors.
      * **Measure:** Total time taken for the search (in milliseconds).
      * **Store:** The resulting indices as `I_exact`.
   
   
   3. **Experiment B: Approximate Search (IVF)**
      * **Theory:** IVF (Inverted File) clusters data into `nlist` cells (Voronoi cells). It needs a "quantizer" (another index) to decide which cell a vector belongs to.
      * **Architecture:** Use `faiss.IndexIVFFlat`.
      * It requires: `quantizer` (use `IndexFlatL2(d)`), `d`, `nlist` (use 100), and `metric` (use `faiss.METRIC_L2`).
      * **Training:** Unlike Flat, IVF **must be trained** to learn the cluster centroids. Train on `xb` *before* adding data.
      * **Add:** Add `xb` to the index.
      * **Search:** Search for `k=1` neighbor.
      * **Measure:** Total time taken (in ms).
      * **Store:** The resulting indices as `I_ivf`.

---

Blueprint for your `approximate` method:
   1. **Define Hyperparameters:**
      * `nlist = 100`: This determines how many "cells" or clusters we divide the space into.
   
   
   2. **Create the Quantizer:**
      * The IVF index needs a "helper" index (called a quantizer) to calculate distances to the cluster centroids.
      * **Syntax:** `quantizer = faiss.IndexFlatL2(d)`
   
   
   3. **Create the IVF Index:**
      * **Syntax:** `index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)`
      * *Note: This tells FAISS "Use this quantizer, for vectors of size d, split into nlist clusters, using L2 distance."*
   
   
   4. **Training (Crucial Step):**
      * Unlike the Flat index, IVF must "learn" where the clusters are before it can store data.
      * **Action:** Call `index.train(xb)`.
      * *Check:* You can print `index.is_trained` to verify it returns `True`.
   
   
   5. **Add and Search:**
      * **Action:** `index.add(xb)` (This assigns every vector to a specific cluster).
      * **Action:** `index.search(xq, k)` (Standard search).
---
   
   4. **Evaluation:**
      * **Recall Calculation:** Calculate what percentage of `I_ivf` matches `I_exact`.
      * Formula: `(matches / nq) * 100`.
      * **Print:**
         * Exact Search Time (ms)
         * IVF Search Time (ms)
         * Recall (%)

**Forbidden Shortcuts:**

* Do not use standard Python lists for calculation; keep everything in NumPy/FAISS.



In [10]:
import numpy as np
import faiss
import time

class Dataset():
    """
    This dataset creates all the classes
    """
    def __init__(self):
        """
        Constructor which creates alls the variables
        """
        self.d = 128
        self.nb = 100000
        self.nq = 100
        self.seed = 42

    def get_vars(self):
        """
        Just get all the eenv vars
        """
        return self.d, self.nb, self.nq, self.seed

    def create_data(self):
        """
        Using the env variables above we create a dataset
        """
        np.random.seed(self.seed)
        xb = np.random.rand(self.nb, self.d).astype(np.float32)
        xq = np.random.rand(self.nq, self.d).astype(np.float32)
        return xb, xq
        

class faiss_index():
    """
    Here, we declare and write the code for all the search indices
    """
    def __init__(self, xb, xq, nq, k, d):
        """
        How many nearest is also declared here
        """
        self.k = k
        self.d = d
        self.xb = xb
        self.xq = xq
        self.nq = nq
        self.I_exact = None
        self.I_ivf = None

    def baseline(self):
        """
        Create a baseline, exact search
        """
        st1 = time.time()
        index = faiss.IndexFlatL2(self.d)
        index.add(self.xb)
        st2 = time.time()
        D, I = index.search(self.xq, self.k)
        et = time.time()
        print(f"Total Time for Exact search = {et - st1}\nTime taken only for search = {et - st2}")
        self.I_exact = I

    def IVF(self):
        st1 = time.time()
        quantizer = faiss.IndexFlatL2(self.d)
        nlist = 100
        index = faiss.IndexIVFFlat(quantizer, self.d, nlist, faiss.METRIC_L2)
        index.train(self.xb)
        print(f"Check if the data is trained...{index.is_trained}")
        index.add(self.xb)
        # index.nprobe = 50
        st2 = time.time()
        D, I = index.search(self.xq, self.k)
        et = time.time()
        print(f"Total Time for IVF  = {et - st1}\nTime taken only for search = {et - st2}")
        self.I_ivf = I

    def calculate_recall(self):
        matches = self.I_exact == self.I_ivf
        recall = (np.sum(matches) / self.nq) * 100
        print(f"Recall = {recall}%")


if __name__ == "__main__":
    dataset = Dataset()
    
    d, nb, nq, seed = dataset.get_vars() 
    xb, xq = dataset.create_data()

    k = 1
    index = faiss_index(xb = xb, xq = xq, nq = nq, k=k, d=d)

    print(f"Dataset: {nb} vectors, {nq} queries, k={k}")

    print("Running Exact (Flat) Search...")
    index.baseline()

    print("\nRunning IVF Search...")
    index.IVF()

    print("\nCalculating Recall...")
    index.calculate_recall()

        

Dataset: 100000 vectors, 100 queries, k=1
Running Exact (Flat) Search...
Total Time for Exact search = 0.11244702339172363
Time taken only for search = 0.08047795295715332

Running IVF Search...
Check if the data is trained...True
Total Time for IVF  = 0.24259066581726074
Time taken only for search = 0.0

Calculating Recall...
Recall = 4.0%



### Code Review

**1. Critique: Logic & Syntax**
   * **Pass:** Your class structure is clean and modular.
   * **Pass:** `k=1` ensures strict neighbor matching.
   * **Pass:** You correctly identified that `IVF` requires training.
   * **Observation:**
   * **Speed:** IVF (15ms) was ~4x faster than Exact (58ms).
   * **Recall:** **4.0%**. This seems disastrously low. This is **not a bug in your code**, but a characteristic of the default IVF settings.


**2. Optimize: The "Pro" Insight (Fixing the 4%)**

Why was recall only 4%?

   * **The Problem:** IVF partitions space into 100 cells (`nlist=100`). By default, FAISS only checks **1 cell** (`nprobe=1`)—the one closest to the query.
   * **The Reality:** In high-dimensional space (128D), the "true" nearest neighbor is often just across the border in a neighboring cell. If you don't check neighbors, you miss it.
   * **The Fix:** Increase `nprobe` (number of cells to visit).

**Try adding this single line inside your `IVF` method, right before the search:**

```python
    def IVF(self):
        # ... (setup and train) ...
        index.add(self.xb)
        
        # PRO TIP: Check the 10 closest cells, not just 1
        index.nprobe = 10  
        
        st2 = time.time()
        D, I = index.search(self.xq, self.k)
        # ...

```

*If you ran this, your Recall would likely jump up, while speed would only drop slightly.*

**3. Rate: PASS**
You have successfully implemented the "Hello World" of Vector Search and proved the speed/accuracy trade-off.

